In [0]:
# Importação das bibliotecas necessárias no código 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, max, lit , month, year
from pyspark.sql.types import DateType

In [0]:
# Inicializa uma Spark Session
spark = SparkSession.builder \
    .appName("credit_card_transactions") \
    .getOrCreate()


In [0]:
# leitura de um arquivo que não está salvo como uma table

file_location = "lugar onde o arquivo está"
file_type = "tipo do arquivo"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# leitura de uma tabela
df = spark.read.table("hive_metastore.default.credit_card_transactions")

display(df)

In [0]:
# 1. Qual a maior transação que houve em um determinado dia
df_temp = df.withColumn("transaction_date", col("transaction_date").cast(DateType()))
max_transaction_per_day = df_temp.groupBy(col("transaction_date")).max("transaction_amount")

display(max_transaction_per_day)

In [0]:
# 2. Qual a maior transação que houve em um determinado mês e ano
df_temp = df.withColumn("transaction_date", col("transaction_date").cast(DateType()))
max_transaction_per_month_year = df_temp.groupBy(month(col("transaction_date")), year(col("transaction_date"))).max("transaction_amount")

display(max_transaction_per_month_year)

In [0]:
# 3. Soma de valor de transação por cliente
sum_transaction_per_customer = df.groupBy("customer_id").sum("transaction_amount")

display(sum_transaction_per_customer)

In [0]:
# 4. Criação de coluna nova (exemplo: adicionar uma coluna de imposto de 10% sobre o valor da transação)
df_with_tax = df.withColumn("transaction_tax", col("transaction_amount") * 0.10)

display(df_with_tax)

In [0]:
# 5. Seleção de colunas específicas (exemplo: selecionar apenas id da transação, id do cliente e valor da transação)
selected_columns = df.select("transaction_id", "customer_id", "transaction_amount")

display(selected_columns)

In [0]:
# 6. Filtragem de datas (exemplo: transações após 2024-11-01)
filtered_df = df.filter(col("transaction_date") > "2024-11-01")

display(filtered_df)

In [0]:
# criando uma view que só pode ser usada no contexto do notebook atual

temp_table_name = "credit_card_transactions_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

select * from hive_metastore.default.credit_card_transactions

In [0]:
# quero salvar em parquet somente quais são os customers id

df = df.select('customer_id').distinct()


In [0]:
permanent_table_name = "credit_card_customers_id"

df.write.format("parquet").saveAsTable(permanent_table_name)